In [27]:
!git clone https://github.com/ngoxuanphong/ENV.git

Cloning into 'ENV'...
remote: Enumerating objects: 1250, done.
remote: Counting objects: 100% (476/476), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 1250 (delta 192), reused 393 (delta 133), pack-reused 774
Receiving objects: 100% (1250/1250), 58.04 MiB | 12.55 MiB/s, done.
Resolving deltas: 100% (502/502), done.


In [28]:
%cd ENV

/content/ENV/ENV/ENV/ENV/ENV


In [29]:

# Check hệ thống

# import Base.MachiKoro.env as env
# from CheckEnv import check_env
# print(check_env(env))

import numpy as np
import random as rd
from numba import njit, jit
import sys, os
from setup import SHORT_PATH
import importlib.util
game_name = 'Durak'

In [30]:

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2


In [31]:
per =np.zeros((0))

In [32]:
@njit()
def StupidBot(state, per):



    ValidAction = getValidActions(state)
    ValidAction = np.where(ValidAction ==1)[0]
    returnAction = -1

    trumpSuit = np.where(state[158:162] == 1)[0][0]
    

    for i in range(13):
        for j in range(4):
            if i + 13*j in ValidAction and j != trumpSuit:
                  returnAction = i + 13* j
                  break
        if returnAction != -1:
            break
    if state[157] == 1:
        attackCard = np.where(state[104:156] == 1)[0][0]
        if attackCard // 13 == trumpSuit and np.sum(state[52:104]) <= 4 and np.min(state[163:166]) >= 3:
            returnAction = 52
        elif attackCard % 13 >= 7 and np.sum(state[52:104]) <= 6 and state[162] > 0:
            returnAction = 52
        elif state[162] > 0:
            returnAction = -1
            for i in range(7,13):
                    for j in range(4):
                        if i + 13*j in ValidAction and j != trumpSuit:
                              returnAction = i + 13* j
                              break
                    if returnAction != -1:
                        break
            
            if returnAction == -1:
                for i in range(7):
                    for j in range(4):
                        if i + 13*j in ValidAction and j != trumpSuit:
                              returnAction = i + 13* j
                              break
                    if returnAction != -1:
                        break


    if state[156] == 1:

            if returnAction != 52 and 52 in ValidAction:
                if (returnAction % 13 >= 11 and returnAction // 13 == trumpSuit ) and state[162] > 0:
                    returnAction = 52
 

    if returnAction == -1:
        returnAction = ValidAction[0]
    

    # print('My cards:',np.where(state[0:52] == 1)[0])
    # print('Defend card:',np.where(state[52:104] == 1)[0])
    # print('Attack card:',np.where(state[104:156] == 1)[0])
    # print('Attack - defend:',state[156:158])
    # print('Trump suit:',state[158:162], trumpSuit)
    # print('Left Card: Deck and three rival:',state[162:166])
    # print(ValidAction, returnAction)
    # print('----------------')

    return returnAction, per

In [33]:
for i in range(10):
    win, per = numba_main_2(StupidBot,10000,per,1)
    print(i, win)

0 5188
1 5162
2 5090
3 5101
4 5183
5 5182
6 5103
7 5163
8 5039
9 5180
